In [1]:
from dbmodels import *
from sqlalchemy import create_engine

In [2]:
import psycopg2
from sqlalchemy.orm import sessionmaker

In [3]:
Restaurant()

<Restaurant(name='None', url='None')>

In [4]:
dbname = 'restaurants'
username = 'andylane'
engine = create_engine('postgres://%s@localhost/%s'%(username,dbname))
print(engine.url)

postgres://andylane@localhost/restaurants


In [6]:
con = None
con = psycopg2.connect(database = "restaurants", user = "andylane")
Session = sessionmaker(bind=engine)
session = Session()

In [7]:
from whoosh.analysis import StemmingAnalyzer
import flask.ext.whooshalchemy

# set the location for the whoosh index
app.config['WHOOSH_BASE'] = './whoosh/'
# set the global analyzer, defaults to StemmingAnalyzer.
app.config['WHOOSH_ANALYZER'] = StemmingAnalyzer()

/home/andylane/anaconda2/envs/insight/lib/python2.7/site-packages/flask/exthook.py:71: ExtDeprecationWarning: Importing flask.ext.whooshalchemy is deprecated, use flask_whooshalchemy instead.
  .format(x=modname), ExtDeprecationWarning
/home/andylane/anaconda2/envs/insight/lib/python2.7/site-packages/flask/exthook.py:71: ExtDeprecationWarning: Importing flask.ext.sqlalchemy is deprecated, use flask_sqlalchemy instead.
  .format(x=modname), ExtDeprecationWarning


NameError: name 'app' is not defined

In [35]:
from flask import Flask

In [73]:
def rebuild_index(model):
    """Rebuild search index of Flask-SQLAlchemy model"""
    #log("Rebuilding {0} index...".format(model.__name__))
    primary_field = "id"
    searchables = model.__searchable__
    index_writer = flask.ext.whooshalchemy.whoosh_index(Flask("views"), model)

    # Fetch all data
    entries = session.query(Restaurant).all()
    entry_count = 0
    with index_writer.writer() as writer:
        for entry in entries:
            index_attrs = {}
            for field in searchables:
                index_attrs[field] = unicode(getattr(entry, field))
            index_attrs[primary_field] = unicode(getattr(entry, primary_field))
            writer.update_document(**index_attrs)
            entry_count += 1
    #log("Rebuilt {0} {1} search index entries.".format(str(entry_count), model.__name__))

In [77]:
rebuild_index(Restaurant)

In [12]:
import os, os.path
from whoosh import index

if not os.path.exists("restaurant_index"):
    os.mkdir("restaurant_index")

ix = index.create_in("restaurant_index", Restaurant)

FieldConfigurationError: <class 'dbmodels.Restaurant'> is not a Schema

In [56]:
from whooshalchemy import IndexService
config = {"WHOOSH_BASE": "./whoosh_index"}
index_service = IndexService(config=config, session=session)
index_service.register_class(Restaurant)

FileIndex(FileStorage('./whoosh_index/Restaurant'), 'MAIN')

In [58]:
session.commit()

In [59]:
Base.metadata.create_all(engine)

In [89]:
Restaurant.search_query("tomato shop").all()[0].name

u'Miss Tomato Sandwich Shop'